# Laboratorium 7

Celem siĂłdmego laboratorium jest zapoznanie siÄ oraz zaimplementowanie algorytmu gĹÄbokiego uczenia aktywnego - Actor-Critic. Zaimplementowany algorytm bÄdzie testowany z wykorzystaniem Ĺrodowiska z OpenAI - _CartPole_.


DoĹÄczenie standardowych bibliotek


In [1]:
from collections import deque
import gym
import numpy as np
import random

ModuleNotFoundError: No module named 'gym'

DoĹÄczenie bibliotek do obsĹugi sieci neuronowych


In [1]:
import torch
import torch.nn as nn

## Zadanie 1 - Actor-Critic

<p style='text-align: justify;'>
Celem Äwiczenie jest zaimplementowanie algorytmu Actor-Critic. W tym celu naleĹźy utworzyÄ dwie gĹÄbokie sieci neuronowe:
    1. *actor* - sieÄ, ktĂłra bÄdzie uczyĹa siÄ optymalnej strategii (podobna do tej z laboratorium 6),
    2. *critic* - sieÄ, ktĂłra bÄdzie uczyĹa siÄ funkcji oceny stanu (podobnie jak siÄ DQN).
Wagi sieci *actor* aktualizowane sÄ zgodnie ze wzorem:
\begin{equation*}
    \theta \leftarrow \theta + \alpha \delta_t \nabla_\theta log \pi_{\theta}(a_t, s_t | \theta).
\end{equation*}
Wagi sieci *critic* aktualizowane sÄ zgodnie ze wzorem:
\begin{equation*}
    w \leftarrow w + \beta \delta_t \nabla_w\upsilon(s_{t + 1}, w),
\end{equation*}
gdzie:
\begin{equation*}
    \delta_t \leftarrow r_t + \gamma \upsilon(s_{t + 1}, w) - \upsilon(s_t, w).
\end{equation*}
</p>


In [ ]:
class REINFORCEAgent:
    def __init__(self, state_size, action_size, actor, critic):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99    # discount rate
        self.learning_rate = 0.001
        self.actor = actor
        self.critic = critic #critic network should have only one output


    def get_action(self, state):
        """
        Compute the action to take in the current state, basing on policy returned by the network.

        Note: To pick action according to the probability generated by the network
        """

        #
        # INSERT CODE HERE to get action in a given state
        #        
        
        return chosen_action

  

    def learn(self, state, action, reward, next_state, done):
        """
        Function learn networks using information about state, action, reward and next state. 
        First the values for state and next_state should be estimated based on output of critic network.
        Critic network should be trained based on target value:
        target = r + \gamma next_state_value if not done]
        target = r if done.
        Actor network shpuld be trained based on delta value:
        delta = target - state_value
        """
        #
        # INSERT CODE HERE to train network
        #
        


Czas przygotowaÄ model sieci, ktĂłra bÄdzie siÄ uczyĹa dziaĹania w Ĺrodowisku [_CartPool_](https://gym.openai.com/envs/CartPole-v0/):


In [ ]:
env = gym.make("CartPole-v0").env
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
alpha_learning_rate = 0.0001
beta_learning_rate = 0.0005

class ActorCriticModel(nn.Module):
    def __init__(self, action_size, state_size) -> None:
        super().__init__()
        self.lin1 =  nn.Linear(action_size, 32)
        self.lin2_1 = nn.Linear(32, state_size)
        self.lin2_2 = nn.Linear(32, state_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.lin1(x))
        out1 = self.lin2_1(x)
        out2 = self.lin2_2(x)
        return out1, out2

Czas nauczyÄ agenta gry w Ĺrodowisku _CartPool_:


In [ ]:
agent = Agent(state_size, action_size, actor_model, critic_model)


for i in range(100):
    score_history = []

    for i in range(100):
        done = False
        score = 0
        state = env.reset()
        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, _ = env.step(action)
            agent.learn(state, action, reward, next_state, done)
            state = next_state
            score += reward
        score_history.append(score)

    print("mean reward:%.3f" % (np.mean(score_history)))

    if np.mean(score_history) > 300:
        print("You Win!")
        break